# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,imbituba,-28.2400,-48.6703,61.09,65,35,9.91,BR,1696280066
1,bilibino,68.0546,166.4372,16.95,96,100,1.52,RU,1696280067
2,port-aux-francais,-49.3500,70.2167,35.58,70,85,34.09,TF,1696279814
3,jamestown,42.0970,-79.2353,81.01,56,20,3.44,US,1696279945
4,la'ie,21.6477,-157.9253,85.10,70,4,3.00,US,1696280069


In [3]:
city_data_df.count()

City          575
Lat           575
Lng           575
Max Temp      575
Humidity      575
Cloudiness    575
Wind Speed    575
Country       570
Date          575
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",

    #https://holoviews.org/user_guide/Style_Mapping.html
    #to change the opacity of the dots
    alpha = [0.6],
    color = "City"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] > 68) & (city_data_df["Max Temp"] < 80)]

# Drop any rows with null values
ideal_weather_df.dropna(how = "any")

# Display sample data
ideal_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
7,adamstown,-25.0660,-130.1015,68.29,72,69,23.73,PN,1696279836
11,albany,42.6001,-73.9662,76.73,52,32,1.99,US,1696279889
16,ponta delgada,37.7333,-25.6667,68.72,73,20,12.66,PT,1696280078
19,west island,-12.1568,96.8225,76.98,73,40,19.57,CC,1696280024
21,lima duarte,-21.8425,-43.7931,68.31,98,79,3.09,BR,1696280082


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
hotel_df = ideal_weather_df[["City","Country", "Lat", "Lng"]].copy()

hotel_df.head()

,City,Country,Lat,Lng
City_ID,,,,
7,adamstown,PN,-25.0660,-130.1015
11,albany,US,42.6001,-73.9662
16,ponta delgada,PT,37.7333,-25.6667
19,west island,CC,-12.1568,96.8225
21,lima duarte,BR,-21.8425,-43.7931


In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City","Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
City_ID,,,,,
7,adamstown,PN,-25.0660,-130.1015,
11,albany,US,42.6001,-73.9662,
16,ponta delgada,PT,37.7333,-25.6667,
19,west island,CC,-12.1568,96.8225,
21,lima duarte,BR,-21.8425,-43.7931,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found


albany - nearest hotel: No hotel found
ponta delgada - nearest hotel: Hotel Talisman
west island - nearest hotel: Cocos Village Bungalows
lima duarte - nearest hotel: No hotel found
kasongo-lunda - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
avarua - nearest hotel: Paradise Inn
civitavecchia - nearest hotel: Hotel San Giorgio Civitavecchia
valley city - nearest hotel: AmericInn
belmonte - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
flying fish cove - nearest hotel: Christmas Island Lodge
bethel - nearest hotel: Hampton Inn Danbury
la sarre - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
kongolo - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
kununurra - nearest hotel: Kununurra Country Club Resort
ribeira grande - nearest hotel: Hospedaria JSF
leo - nearest hotel: Auberge le prestige
bonthe - nearest hotel: No hotel found
bamboo flat - nearest hotel: Hotel Meg

,City,Country,Lat,Lng,Hotel Name
City_ID,,,,,
7,adamstown,PN,-25.0660,-130.1015,No hotel found
11,albany,US,42.6001,-73.9662,No hotel found
16,ponta delgada,PT,37.7333,-25.6667,Hotel Talisman
19,west island,CC,-12.1568,96.8225,Cocos Village Bungalows
21,lima duarte,BR,-21.8425,-43.7931,No hotel found
...,...,...,...,...,...
556,aguilas,ES,37.4063,-1.5829,Hotel El Paso
560,tianpeng,CN,30.9866,103.9393,雅阁酒店
565,saint-leu,RE,-21.1500,55.2833,Iloha Seaview Hôtel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    alpha = [0.6],
    color = "City",
    #https://hvplot.holoviz.org/reference/geopandas/points.html
    #to add hover info
    hover_cols = ["Hotel Name", "City"]

)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)